# STARSKØPE

## Part III: Restricted Boltzmann Machines


WORK IN PROGRESS 

Bernoulli Restricted Boltzmann Machine (RBM).

    A Restricted Boltzmann Machine with binary visible units and
    binary hidden units. Parameters are estimated using Stochastic Maximum
    Likelihood (SML), also known as Persistent Contrastive Divergence (PCD)
    [2].

    The time complexity of this implementation is ``O(d ** 2)`` assuming
    d ~ n_features ~ n_components.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1ZRUudjsf0ofOMVjhzfF239Df3BaO2jqp#scrollTo=n0aJxtzZMCSI&uniqifier=4)

In [2]:
import numpy as np
import pandas as pd
import matplotlib as mpl
%matplotlib inline
from matplotlib.colors import LogNorm
import matplotlib.pyplot as plt

import seaborn as sns
sns.set_style('whitegrid')
import matplotlib.pyplot as plt
plt.style.use('seaborn-bright')
 

font_dict={'family':'"Titillium Web", monospace','size':16}
mpl.rc('font',**font_dict)


#ignore pink warnings
import warnings
warnings.filterwarnings('ignore')
# Allow for large # columns
pd.set_option('display.max_columns', 0)
# pd.set_option('display.max_rows','')


from scipy.ndimage import convolve
from sklearn import linear_model, datasets, metrics
from sklearn.model_selection import train_test_split
from sklearn.neural_network import BernoulliRBM
from sklearn.pipeline import Pipeline
from sklearn.base import clone

In [ ]:
# Use this instead if using Jupyter notebook/local
# import os, glob, sys
# home = os.path.abspath(os.curdir)
# os.listdir(home)
# %cd ../
# %cd data/
# %ls

In [3]:
import os, glob, sys
from os import path

base_folder = r'specs/'
os.listdir(base_folder)

['test', 'train']

In [4]:
train_base_dir = base_folder+'train/'
test_base_dir = base_folder+'test/'

train_planets_dir = train_base_dir+'planets/'
train_negatives_dir = train_base_dir+'negatives/'

test_planets_dir = test_base_dir+'planets/'
test_negatives_dir = test_base_dir+'negatives/'

In [5]:
planet_train_files = glob.glob(train_planets_dir+'*.png')
negative_train_files = glob.glob(train_negatives_dir+'*.png')
all_train_files = [*planet_train_files, *negative_train_files]

planet_test_files = glob.glob(test_planets_dir+'*.png')
negative_test_files = glob.glob(test_negatives_dir+'*.png')
all_test_files = [*planet_test_files, *negative_test_files]

all_filename_vars = [planet_train_files, negative_train_files,
                     planet_test_files, negative_test_files]

In [6]:
def load_img_cv2(filename, RGB=True):
    import cv2
    IMG = cv2.imread(filename)
    
    if RGB: 
        cmap = cv2.COLOR_BGR2RGB
    
    else:
        cmap=cv2.COLOR_BGR2GRAY
        
    return cv2.cvtColor(IMG, cmap)


In [7]:
from PIL import Image
from keras.preprocessing import image
from imageio import imread
from skimage.transform import resize
from tqdm import tqdm

# defining a function to read images and convert to array
def read_img(img_path, target_size=(8,8)):
    #img = load_img_cv2(img_path)
    #img = image.img_to_array(img)
    img = image.load_img(img_path, target_size=target_size, interpolation='bilinear')
    img = image.img_to_array(img)
    return img

Using TensorFlow backend.


In [8]:
# Read in training and test filenames to produce X and y data splits.

def xy_arrays(planet_train_filenames, negative_train_filenames, 
              planet_test_filenames, negative_test_filenames, 
              img_size=(8,8)):
    # y labels are encoded as 0=negatives, 1=planets
    # returns X_train, X_test, y_train, y_test, y_val
    
    import numpy as np
    
    display('[i] LOADING IMAGES')
    X_img = []
    y_label = []
    
    for img_path in tqdm(planet_train_files):
        X_img.append(read_img(img_path, target_size=img_size))
        y_label.append(1)
        
    for img_path in tqdm(negative_train_files):
        X_img.append(read_img(img_path, target_size=img_size))
        y_label.append(0)
    
    for img_path in tqdm(planet_test_files):
        X_img.append(read_img(img_path, target_size=img_size))
        y_label.append(1)
        
    for img_path in tqdm(negative_test_files):
        X_img.append(read_img(img_path, target_size=img_size))
        y_label.append(0)
        
    from sklearn.model_selection import train_test_split
    X = np.array(X_img, np.float32)
    y = np.array(y_label)
    
    return X, y

In [11]:
X, y = xy_arrays(planet_train_files, negative_train_files, 
              planet_test_files, negative_test_files, 
              img_size=(64,64,3))

'[i] LOADING IMAGES'

100%|██████████| 5/5 [00:00<00:00, 48.90it/s]


In [12]:
X.shape, y.shape

((84, 64, 64, 3), (84,))

In [15]:
X[0][1]

array([[254., 229., 222.],
       [251., 150., 119.],
       [251., 143., 108.],
       [157.,  92.,  74.],
       [ 20.,  20.,  24.],
       [ 22.,  20.,  25.],
       [163.,  73.,  79.],
       [237., 100., 107.],
       [238., 104., 107.],
       [248., 130., 106.],
       [250., 136., 106.],
       [242., 120., 108.],
       [228.,  91., 113.],
       [228.,  90., 113.],
       [220.,  84., 117.],
       [214.,  81., 120.],
       [211.,  80., 121.],
       [184.,  69., 131.],
       [177.,  66., 134.],
       [186.,  70., 130.],
       [218.,  84., 118.],
       [221.,  85., 117.],
       [215.,  82., 120.],
       [208.,  77., 124.],
       [207.,  77., 124.],
       [171.,  64., 133.],
       [153.,  59., 138.],
       [156.,  59., 137.],
       [175.,  66., 134.],
       [178.,  66., 133.],
       [192.,  72., 128.],
       [215.,  81., 120.],
       [214.,  81., 120.],
       [180.,  68., 130.],
       [159.,  60., 137.],
       [166.,  65., 134.],
       [230., 106., 109.],
 

In [159]:
# Authors: Yann N. Dauphin, Vlad Niculae, Gabriel Synnaeve
# License: BSD

# #############################################################################
# Setting up

def nudge_dataset(X, Y):
    """
    This produces a dataset 5 times bigger than the original one,
    by moving the 8x8 images in X around by 1px to left, right, down, up
    """
    direction_vectors = [
        [[0, 1, 0],
         [0, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [1, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [0, 0, 1],
         [0, 0, 0]],

        [[0, 0, 0],
         [0, 0, 0],
         [0, 1, 0]]]

    def shift(x, w):
        return convolve(x.reshape((8, 8)), mode='constant', weights=w).ravel()

    X = np.concatenate([X] +
                       [np.apply_along_axis(shift, 1, X, vector)
                        for vector in direction_vectors])
    Y = np.concatenate([Y for _ in range(5)], axis=0)
    return X, Y

In [160]:
X, Y = nudge_dataset(X, y)

In [170]:
X.shape

(420, 64, 64, 3)

In [161]:
Y

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [162]:
X = (X - np.min(X, 0)) / (np.max(X, 0) + 0.0001)  # 0-1 scaling

In [163]:
X.shape, y.shape

((420, 64, 64, 3), (84,))

In [ ]:
# X_train,X_test,X_val,y_train,y_test,y_val=load_train_test_val(planet_train_files, 
#                                                               negative_train_files,
#                                                               planet_test_files, 
#                                                               negative_test_files, 
#                                                               val_size=0.1, 
#                                                               img_size=(64,64,3))


#     X_train,X_val, y_train, y_val = train_test_split(X,y,test_size=0.1)
    
#     print('\n[i] Length of Splits:')
#     print(f"X_train={len(X_train)}, X_test={len(X_test)}, X_val={len(X_val)}")
    
#     return X_train, X_test, X_val, y_train, y_test, y_val

In [164]:
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.2, random_state=0)

In [166]:
# Models we will use
logistic = linear_model.LogisticRegression(solver='newton-cg', tol=1)
rbm = BernoulliRBM(random_state=0, verbose=True)

rbm_features_classifier = Pipeline(
    steps=[('rbm', rbm), ('logistic', logistic)])

In [167]:
# #############################################################################
# Training

# Hyper-parameters. These were set by cross-validation,
# using a GridSearchCV. Here we are not performing cross-validation to
# save time.
rbm.learning_rate = 0.06
rbm.n_iter = 10
# More components tend to give better prediction performance, but larger
# fitting time
rbm.n_components = 100
logistic.C = 6000

In [169]:
# Training RBM-Logistic Pipeline
rbm_features_classifier.fit(X_train, Y_train)

ValueError: Found array with dim 4. Estimator expected <= 2.

In [138]:
# Training the Logistic regression classifier directly on the pixel
raw_pixel_classifier = clone(logistic)
raw_pixel_classifier.C = 100.
raw_pixel_classifier.fit(X_train, Y_train)

ValueError: Found array with dim 4. Estimator expected <= 2.

In [ ]:
# #############################################################################
# Evaluation

Y_pred = rbm_features_classifier.predict(X_test)
print("Logistic regression using RBM features:\n%s\n" % (
    metrics.classification_report(Y_test, Y_pred)))

Y_pred = raw_pixel_classifier.predict(X_test)
print("Logistic regression using raw pixel features:\n%s\n" % (
    metrics.classification_report(Y_test, Y_pred)))

In [ ]:
# #############################################################################
# Plotting

plt.figure(figsize=(4.2, 4))
for i, comp in enumerate(rbm.components_):
    plt.subplot(10, 10, i + 1)
    plt.imshow(comp.reshape((8, 8)), cmap=plt.cm.gray_r,
               interpolation='nearest')
    plt.xticks(())
    plt.yticks(())
plt.suptitle('100 components extracted by RBM', fontsize=16)
plt.subplots_adjust(0.08, 0.02, 0.92, 0.85, 0.08, 0.23)

plt.show()